In [1]:
from datasets import load_dataset
import numpy as np
import torch

/home/andoni.sudupe/.conda/envs/omni/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import joblib
import torch
from transformers import Wav2Vec2Processor, HubertModel
from datasets import load_dataset
import numpy as np
from torchaudio import load, transforms

/home/andoni.sudupe/.conda/envs/omni/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_repo = "Ansu/mHubert-basque-k1000-L9"
processor = Wav2Vec2Processor.from_pretrained(model_repo)
model = HubertModel.from_pretrained(model_repo)
model.eval()

HubertModel(
  (feature_extractor): HubertFeatureEncoder(
    (conv_layers): ModuleList(
      (0): HubertGroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x HubertNoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x HubertNoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): HubertFeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): HubertEncoder(
    (pos_conv_embed): HubertPositionalConvEmbedding(
      (conv): Para

In [11]:
audio_path = "/scratch/andoni.sudupe/Instruct_S2S_eu/answers/instruct_eu_0_assistant_0.wav"
answer, old_sr = load(audio_path)
sr = 16000
answer = transforms.Resample(orig_freq=old_sr, new_freq=sr)(answer)
answer = answer.squeeze(0)

In [12]:
answer.shape

torch.Size([147375])

In [15]:
inputs.to('cpu')

{'input_values': tensor([[-0.0323, -0.0332, -0.0273,  ..., -0.0123, -0.0123, -0.0123]])}

In [16]:
inputs = processor(answer, sampling_rate=sr, return_tensors="pt", padding=True)
inputs = inputs.to("cuda")
with torch.no_grad():
    out = model(**inputs, output_hidden_states=True)
features = out.hidden_states[9].squeeze(0).cpu().numpy()

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [1]:
from datasets import load_from_disk
dataset = load_from_disk("/scratch/andoni.sudupe/Instruct_S2U_eu/hf")

/home/andoni.sudupe/.conda/envs/omni/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [54]:
from tqdm import tqdm

In [59]:
len(ids)

147

In [2]:
df = dataset.to_pandas()
print('converted')
df = df[df["answer_token"].str.len() >= 2]

converted


In [3]:
filtered_dataset = dataset.from_pandas(df)

In [4]:
dataset

Dataset({
    features: ['id', 'round', 'question', 'answer', 'answer_token', 'question_audio'],
    num_rows: 420734
})

In [5]:
filtered_dataset

Dataset({
    features: ['id', 'round', 'question', 'answer', 'answer_token', 'question_audio', '__index_level_0__'],
    num_rows: 278617
})

In [60]:
def filter_fn(example):
    return len(example["answer_token"]) >= 2

filtered_dataset = dataset.filter(filter_fn)

Filter:   0%|          | 0/420734 [00:00<?, ? examples/s]

Filter:   1%|          | 3000/420734 [01:13<2:51:30, 40.60 examples/s]


KeyboardInterrupt: 

In [25]:
torch.tensor(dataset[0]['question_audio']).shape

torch.Size([1, 54740])

In [6]:
dataset.save_to_disk("/scratch/andoni.sudupe/Instruct_S2S_eu/hf")

Saving the dataset (323/323 shards): 100%|██████████| 420734/420734 [26:12<00:00, 267.57 examples/s]


In [5]:
dataset[0]['question_audio']

[[0.0009102446492761374,
  0.0014968885807320476,
  0.001725339563563466,
  0.0017418646020814776,
  0.001703303074464202,
  0.0016767914639785886,
  0.0017704929923638701,
  0.0016736487159505486,
  0.0017179758287966251,
  0.0016760635189712048,
  0.0016923209186643362,
  0.001759821898303926,
  0.0017823552479967475,
  0.001808684435673058,
  0.0018727283459156752,
  0.0018939346773549914,
  0.0019535538740456104,
  0.0020844759419560432,
  0.0020830973517149687,
  0.0020600215066224337,
  0.002190839033573866,
  0.002226976677775383,
  0.002220637397840619,
  0.0022890630643814802,
  0.002295676851645112,
  0.002200729912146926,
  0.0021947177592664957,
  0.0022448543459177017,
  0.0022744201123714447,
  0.0023130548652261496,
  0.0022575196344405413,
  0.0022456934675574303,
  0.002317846054211259,
  0.0022699085529893637,
  0.0022459449246525764,
  0.0022613126784563065,
  0.0022752878721803427,
  0.002339164959266782,
  0.002380135003477335,
  0.0023649896029382944,
  0.00239651

In [37]:
torch.tensor(a).shape

torch.Size([1, 54740])

In [3]:
from omni_speech.model.builder import create_model
import argparse
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchaudio import load
from omni_speech.conversation import conv_templates
import math
import json
from omni_speech.datasets.preprocess import tokenizer_speech_token
from transformers import TrainingArguments, Trainer, TrainerCallback
from torch.nn.utils.rnn import pad_sequence
import torch.distributed as dist
import gc

In [4]:
def collate_fn(batch):
    """Optimized collate function with memory management"""
    input_ids, labels, speech_tensors, speech_lengths = zip(
        *[list(item.values()) for item in batch]
    )
    
    # Pad sequences
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=128009)
    labels = pad_sequence(labels, batch_first=True, padding_value=128009)

    # Handle speech tensors more efficiently
    max_length = max(t.shape[-1] for t in speech_tensors)
    batch_size = len(speech_tensors)
    
    # Pre-allocate tensor to avoid fragmentation
    padded_speech = torch.zeros(batch_size, max_length, dtype=torch.float32)
    
    for i, tensor in enumerate(speech_tensors):
        length = tensor.shape[-1] if tensor.dim() > 1 else tensor.shape[0]
        if tensor.dim() == 2 and tensor.shape[0] == 1:
            padded_speech[i, :length] = tensor.squeeze(0)
        else:
            padded_speech[i, :length] = tensor
    
    # Convert to bfloat16 only when needed
    speech_lengths = torch.tensor(speech_lengths, dtype=torch.long)
    
    return {
        "input_ids": input_ids,
        "labels": labels, 
        "speech": padded_speech.to(torch.bfloat16),
        "speech_lengths": speech_lengths
    }

class CustomDataset(Dataset):
    def __init__(self, questions, tokenizer, conv_mode, model_config, input_type, mel_size, device):
        self.questions = questions
        self.tokenizer = tokenizer
        self.conv_mode = conv_mode
        self.model_config = model_config
        self.input_type = input_type
        self.mel_size = mel_size
        self.device = device
    def __getitem__(self, index):
        item = self.questions[index]

        qs = item['question']
        re = item['answer']
    # def __getitem__(self, index):
    #     item = self.questions[index]
    #     speech_file = item["speech"]
    #     qs = item["conversations"][0]["value"]
    #     re = item["conversations"][1]["value"]

        conv = conv_templates[self.conv_mode].copy()
        conv.append_message(conv.roles[0], "<speech>\nPlease directly answer the questions in the user's speech.")
        conv.append_message(conv.roles[1], re)
        prompt = conv.get_prompt()

        speech = torch.tensor(item["question_audio"]).squeeze(0)

        max_audio_length = 480000  # ~30 seconds at 16kHz, adjust as needed
        if speech.shape[0] > max_audio_length:
            speech = speech[:max_audio_length]
        
        input_ids = tokenizer_speech_token(prompt, self.tokenizer, return_tensors='pt')
        input_ids = input_ids.squeeze(0)  # Remove batch dimension for collate_fn
        
        return {
            "input_ids": input_ids,
            "labels": input_ids.clone(),
            "speech": speech,
            "speech_lengths": speech.shape[0]
        }
    #     try:
    #         # Load and process audio more efficiently
    #         speech, sample_rate = load(speech_file)
    #         speech = speech.squeeze(0)
            
    #         # Truncate very long audio to prevent OOM
    #         max_audio_length = 480000  # ~30 seconds at 16kHz, adjust as needed
    #         if speech.shape[0] > max_audio_length:
    #             speech = speech[:max_audio_length]
            
    #         input_ids = tokenizer_speech_token(prompt, self.tokenizer, return_tensors='pt')
    #         input_ids = input_ids.squeeze(0)  # Remove batch dimension for collate_fn
            
    #         return {
    #             "input_ids": input_ids,
    #             "labels": input_ids.clone(),
    #             "speech": speech,
    #             "speech_lengths": speech.shape[0]
    #         }
    #     except Exception as e:
    #         print(f"Error processing {speech_file}: {e}")
    #         # Return a smaller dummy item
    #         input_ids = tokenizer_speech_token(prompt, self.tokenizer, return_tensors='pt').squeeze(0)
    #         return {
    #             "input_ids": input_ids,
    #             "labels": input_ids.clone(),
    #             "speech": torch.zeros(8000, dtype=torch.float32),  # Smaller dummy
    #             "speech_lengths": 8000
    #         }

    def __len__(self):
        return len(self.questions)

def create_data_loader(questions, tokenizer, conv_mode, model_config, input_type, mel_size, device, batch_size=2, num_workers=1):
    dataset = CustomDataset(questions, tokenizer, conv_mode, model_config, input_type, mel_size, device)
    return dataset

def split_list(lst, n):
    """Split a list into n (roughly) equal-sized chunks"""
    chunk_size = math.ceil(len(lst) / n)
    return [lst[i:i+chunk_size] for i in range(0, len(lst), chunk_size)]

def get_chunk(lst, n, k):
    chunks = split_list(lst, n)
    return chunks[k]

class OptimizedMemoryCallback(TrainerCallback):
    def __init__(self):
        self.step_count = 0
        
    def on_step_end(self, args, state, control, **kwargs):
        self.step_count += 1
        
        # Clear cache every few steps
        if self.step_count % 10 == 0:
            torch.cuda.empty_cache()
            gc.collect()
            
        if torch.cuda.is_available() and self.step_count % 50 == 0:
            for device_id in range(torch.cuda.device_count()):
                allocated = torch.cuda.memory_allocated(device_id) / 1024**3
                reserved = torch.cuda.memory_reserved(device_id) / 1024**3
                print(f"[Step {state.global_step}] GPU {device_id} - Allocated: {allocated:.2f} GB | Reserved: {reserved:.2f} GB")
        
        return control

def train_model(args):
    # Initialize distributed training if using multiple GPUs
    # if 'WORLD_SIZE' in os.environ:
    #     dist.init_process_group(backend='nccl')
    #     local_rank = int(os.environ['LOCAL_RANK'])
    #     torch.cuda.set_device(local_rank)
    #     device = f'cuda:{local_rank}'
    # else:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Set memory fraction to prevent OOM
    # if torch.cuda.is_available():
    #     torch.cuda.set_per_process_memory_fraction(0.95)  # Use 95% of GPU memory
        
    model_path = os.path.expanduser(args.model_path)
    tokenizer, model, context_len = create_model(
        model_path, 
        args.model_base, 
        is_lora=args.is_lora, 
        s2s=args.s2s, 
        device=device,
        # low_cpu_mem_usage=True
    )

    # questions = json.load(open(os.path.expanduser(args.question_file), "r"))
    print("Loading dataset...")
    print(f"Total questions loaded: {len(dataset)}")
    questions = get_chunk(dataset, args.num_chunks, args.chunk_idx)
    data_loader = create_data_loader(
        questions, 
        tokenizer, 
        args.conv_mode, 
        model.config, 
        args.input_type, 
        args.mel_size, 
        device
    )

    # Optimize model for training
    model = model.to(torch.bfloat16)
    
    # Freeze speech encoder to save memory
    for p in model.get_model().speech_encoder.parameters():
        p.requires_grad = False
    
    # Only train the projector
    for p in model.get_model().speech_projector.parameters():
        p.requires_grad = True

    # Set up tokenizer
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

    # Optimized training arguments for H100
    training_args = TrainingArguments(
        output_dir=f'saves/{args.output_folder}',
        overwrite_output_dir=True,
        do_train=True,
        do_eval=False,
        
        # Batch size optimization for H100
        per_device_train_batch_size=1,  # Start small, increase if stable
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=8,  # Effective batch size = 1 * 8 * num_gpus
        
        # Learning rate and optimization
        learning_rate=5e-5,  # Slightly lower for stability
        weight_decay=0.01,
        adam_beta2=0.95,
        warmup_ratio=0.03,
        lr_scheduler_type='cosine',
        
        # Memory optimization
        bf16=True,
        tf32=True,  # Use TF32 on H100 for better performance
        gradient_checkpointing=True,
        dataloader_pin_memory=False,
        dataloader_num_workers=2,  # Reduce if OOM persists
        
        # Distributed training settings
        ddp_find_unused_parameters=False,  # More efficient DDP
        ddp_broadcast_buffers=False,
        
        # Logging and saving
        logging_steps=10,
        save_steps=500,
        save_total_limit=3,
        num_train_epochs=3,
        
        # Reporting
        report_to=['tensorboard', 'wandb'] if args.use_wandb else ['tensorboard'],
        seed=3407,
        
        # Additional optimizations
        remove_unused_columns=False,  # Keep all columns for custom forward
        prediction_loss_only=True,
        
        # Advanced memory settings
        max_grad_norm=1.0,  # Gradient clipping
        group_by_length=False,
    )

    # Clear cache before training
    torch.cuda.empty_cache()
    gc.collect()

    trainer = Trainer(
        model=model,
        processing_class=tokenizer,
        args=training_args,
        train_dataset=data_loader,
        data_collator=collate_fn,
        callbacks=[OptimizedMemoryCallback()],
    )
    
    # Start training with error handling
    try:
        trainer.train()
    except RuntimeError as e:
        if "out of memory" in str(e):
            print("CUDA OOM Error! Try reducing batch size or sequence length.")
            torch.cuda.empty_cache()
            gc.collect()
            raise
        else:
            raise

In [5]:
MODEL_PATH="HiTZ/Latxa-Llama-3.1-8B-Instruct"
QUESTION_FILE="/scratch/andoni.sudupe/Instruct_S2S_eu/data.json"
args = argparse.Namespace(
    model_path=MODEL_PATH,
    model_base=None,
    is_lora=False,
    s2s=False,
    question_file=QUESTION_FILE,
    output_folder="latxa-3.1-8b-instruct-s2s-projector",
    conv_mode="llama_3",
    input_type="raw",
    mel_size=128,
    num_chunks=1,
    chunk_idx=0,
    batch_size=2,
    num_workers=1,
    use_wandb=False
)


In [ ]:
train_model(args)

You are using a model of type llama to instantiate a model of type omni_speech_llama. This is not supported for all configurations of models and can yield errors.
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.11s/it]


Encoder path: Ansu/mHubert-basque-k1000-L9


In [ ]:
!pip 

In [14]:
from transformers import MarianMTModel, MarianTokenizer
model_name = "HiTZ/mt-hitz-en-eu"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name, use_safetensors=True, return_dict=True)

/home/andoni.sudupe/envs/hifi_venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [21]:
src_text = ["Hi, how are you?", "Good, thanks! And you?", "give me 2 examples of machine translation. And also, explain me how to do it."]

translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True), return_dict=True)
print([tokenizer.decode(t, skip_special_tokens=True) for t in translated])


TypeError: MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(8001, 512, padding_idx=8000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(8001, 512, padding_idx=8000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), ep

In [2]:
!pip install unbabel-comet

  Using cached unbabel_comet-2.2.7-py3-none-any.whl.metadata (19 kB)
  Using cached entmax-1.3-py3-none-any.whl.metadata (348 bytes)
  Using cached jsonargparse-3.13.1-py3-none-any.whl.metadata (55 kB)
  Using cached protobuf-4.25.8-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
  Using cached pytorch_lightning-2.5.5-py3-none-any.whl.metadata (20 kB)
  Using cached torchmetrics-0.10.3-py3-none-any.whl.metadata (15 kB)
  Using cached lightning_utilities-0.15.2-py3-none-any.whl.metadata (5.7 kB)
Using cached unbabel_comet-2.2.7-py3-none-any.whl (90 kB)
Using cached jsonargparse-3.13.1-py3-none-any.whl (101 kB)
Using cached entmax-1.3-py3-none-any.whl (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 38.7 MB/s  0:00:00s eta 0:00:01
Using cached protobuf-4.25.8-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
Using cached pytorch_lightning-2.5.5-py3-none-any.whl (832 kB)
Using cached torchmetrics-0.10.3-py3-none-any.whl (529 kB)
Using cached lightning_utilities-0.15.2-py

In [ ]:
from comet import download_model, load_from_checkpoint

# Download and load a quality estimation model (no references needed)
model_path = download_model("Unbabel/wmt22-cometkiwi-da")
model = load_from_checkpoint(model_path)

# Example data (source, hypothesis)
data = [
    {
        "src": "The output signal provides constant sync so the display never glitches.",
        "mt": "Das Ausgangssignal bietet eine konstante Synchronisation, so dass die Anzeige nie stört."
    },
    {
        "src": "Kroužek ilustrace je určen všem milovníkům umění ve věku od 10 do 15 let.",
        "mt": "Кільце ілюстрації призначене для всіх любителів мистецтва у віці від 10 до 15 років."
    },
    {
        "src": "Mandela then became South Africa's first black president after his African National Congress party won the 1994 election.",
        "mt": "その後、1994年の選挙でアフリカ国民会議派が勝利し、南アフリカ初の黒人大統領となった。"
    }
]

# Get prediction
model_output = model.predict(data)

print(f"Confidence score: {model_output}")


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 662.31it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.2 to v2.5.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../home/andoni.sudupe/.cache/huggingface/hub/models--Unbabel--wmt22-cometkiwi-da/snapshots/1ad785194e391eebc6c53e2d0776cada8f83179a/checkpoints/model.ckpt`
Encoder model frozen.
/home/andoni.sudupe/.conda/envs/omni/lib/python3.12/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
/home/andoni.sudupe/.conda/envs/omni/lib/python3.12/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.12 /home/andoni.sudupe/.conda/envs/omni/lib/

Confidence score: s


In [7]:
sys_score.items()

AttributeError: 'str' object has no attribute 'items'

In [ ]:
!huggingface-cli login --token ... --add-to-git-credential

Token is valid (permission: write).
The token `HF_TOKEN` has been saved to /home/andoni.sudupe/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/andoni.sudupe/.cache/huggingface/token
Login successful.
The current active token is: `HF_TOKEN`


In [36]:
!git config --global credential.helper store

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
from datasets import load_dataset

# -------------------------------
# 1. Load models
# -------------------------------

# LLaMA-Instruct model
llm_model_name = "HiTZ/Latxa-Llama-3.1-8B-Instruct"  # example
# tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
# model = AutoModelForCausalLM.from_pretrained(llm_model_name, device_map="auto")
pipe = pipeline(
    "text-generation",
    model=llm_model_name,
    tokenizer=llm_model_name,
    device_map="auto",
    torch_dtype="auto",
)
# tokenizer = AutoTokenizer.from_pretrained("HiTZ/Latxa-Llama-3.1-8B-Instruct")
# model = AutoModelForCausalLM.from_pretrained("HiTZ/Latxa-Llama-3.1-8B-Instruct", device_map="auto", torch_dtype="auto")
# messages = [
#     {"role": "user", "content": "Who are you?"},
# ]

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.06s/it]


In [3]:
examples = [{"eu": "Zeintzuk dira uraren kutsadura gutxitzeko modu batzuk? Uraren kutsadura murrizteko, hondakinak behar bezala ezabatzeko, produktu ekologikoak erabiltzeko eta ura kontserbatzeko, txorrotak itzali behar dira erabiltzen ez direnean.",
        "en": "What are some ways to reduce water pollution? To reduce water pollution, dispose of waste properly, use eco-friendly products, and conserve water by turning off taps when not in use."
},
{"eu": "asdgafghsdfhdaf.",
        "en": "What are some ways to reduce water pollution? To reduce water pollution, dispose of waste properly, use eco-friendly products, and conserve water by turning off taps when not in use."
},
{"eu": "Zeinbbbbbn.",
        "en": "What are some ways to reduce water pollution? To reduce water pollution, dispose of waste properly, use eco-friendly products, and conserve water by turning off taps when not in use."
},
{"eu": "aaaa.",
        "en": "What are some ways to reduce water pollution? To reduce water pollution, dispose of waste properly, use eco-friendly products, and conserve water by turning off taps when not in use."
},
]
# process(example)

In [53]:
# -------------------------------
# 2. Define LLaMA evaluation function
# -------------------------------

def evaluate_translation(src, mt):
    prompt = f"""
You are a bilingual evaluator (English-Basque). Rate the Basque translation below on a scale 0-1
based on: 1) Faithfulness to the English sentence, 2) Preservation of instructions or actions, 3) Is it language dependant.

Answer ONLY with a single number between 0 and 1.

<User>:
"en": "Hi, how are you?",
"eu": "Kaixo, zer moduz?"

<Assistant>:
1.0

<User>:
"en": "Can you tell me 2 interesting facts about whales? ",
"eu": "Baleen buruzko 2 datu interesgarri konta diezazkidazu?"

<Assistant>:
1.0

<User>:
"en": "Can I go to the park?",
"eu": "Parkean joan al naiteke?"

<Assistant>:
0.5

<User>:
"en": "Give me 5 examples of machine translation",
"eu": "Ekarri 5 makina"

<Assistant>:
0.2

<User>:
"en": "Is this sentence correct? I have like this movie",
"eu": "Esaldi hau zuzena al da? Gustatzen zait filma hau"

<Assistant>:
0.0

<User>:
"en": {src}
"eu": {mt}

<Assistant>:
"""
    
    # prompt = pipe.tokenizer.apply_chat_template(
    #         prompt, tokenize=False, add_generation_prompt=True
    #     )
    response = pipe(prompt,         
        max_new_tokens=3,       # keep answers short
        # temperature=0.7,         # fully deterministic
        top_p=1,               # no nucleus sampling
        repetition_penalty=0.7,  # avoid hallucination
        do_sample=False,          # greedy decoding
        return_full_text=False)

    response = float(response[0]['generated_text'])
    return response

# -------------------------------
# 5. Process dataset
# -------------------------------

# dataset = load_dataset("your_dataset_name")

def process(example):
    if len(example['answer_token']) < 2:
        example.update({
            "llm_score": 0.0,
            "valid": False
        })
        return example

    try:
        llm_score = evaluate_translation(example["en"], example["eu"][8:])
        valid = (llm_score >= 0.5)

    except Exception as e:
        print(f"Error processing example {example['id']}: {e}")
        llm_score = 0.0
        valid = False

    # Decide validity: thresholds can be tuned
    
    example.update({
        "llm_score": llm_score,
        "valid": valid
    })
    return example

In [ ]:
results = []
batch_size = 32
for example in tqdm(merged_dataset):
    scores = evaluate_translation(example["en"], example["eu"][8:])
    valids = scores >= 0.5
    results.extend((scores, valids))


  0%|          | 0/420734 [00:00<?, ?it/s]/home/andoni.sudupe/.conda/envs/omni/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
  0%|          | 1586/420734 [02:09<8:20:31, 13.96it/s] 

In [134]:
en_dataset[0]['conversation']

[{'from': 'human',
  'speech': 'data/multiturn_instruction/en/wav/instruct_en_0/instruct_en_0-1-user.wav',
  'text': 'Hey, can you give me, like, three tips for staying healthy?',
  'unit': None},
 {'from': 'gpt',
  'speech': 'data/multiturn_instruction/en/wav/instruct_en_0/instruct_en_0-1-assistant.wav',
  'text': 'Here are three tips for staying healthy: eat a balanced diet, stay hydrated by drinking plenty of water, and get at least seven hours of sleep each night.',
  'unit': '<776><1312><4299><4299><4299><4218><3651><5838><5838><5838><5838><5109><5838><5109><816><4299><5352><4704><2769><5750><6047><6058><1838><2054><4079><1658><2230><6507><6376><3000><4339><6061><5100><2580><2112><4676><4458><4398><4517><59><1917><1920><4752><4914><3834><6534><1928><1610><4509><4672><1356><2247><2408><2195><2438><2438><2609><4319><3509><4632><5106><5114><4862><5093><6372><6537><4194><71><1286><1295><1295><1258><4299><4299><1788><3732><5919><5838><5190><2922><2436><2031><1951><2006><5668><5668><510

In [5]:
from datasets import load_dataset, load_from_disk
from tqdm import tqdm

# Load your datasets
en_dataset = load_dataset("ICTNLP/InstructS2S-200K")["train"]  # or "validation"/"test"

# eu_dataset = load_dataset("Ansu/Instruct_S2U_eu")["train"]
eu_dataset = load_from_disk("/scratch/andoni.sudupe/Instruct_S2U_eu/hf")

# Convert to dict for fast lookup
# print("Creating eu dict...")
# eu_dict = {f"{ex['id'].split('_')[-1]}_{ex['round']}": f"{ex['question']} {ex['answer']}" for ex in tqdm(eu_dataset) if len(ex['answer_token']) > 1}

# en_dict = {f"{ex['id'].split('_')[-1]}_{ex['round']}": f"{ex['question']} {ex['answer']}" for ex in en_dataset}
print("Creating en dict...")
en_dict = {}
for ex in tqdm(en_dataset):
    for round, d in enumerate(range(0,len(ex['conversation']), 2)):
        en_dict[f"{ex['id'].split('_')[-1]}_{round}"] = f"<USER>: {ex['conversation'][d]['text']}\n<ASSISTANT>: {ex['conversation'][d+1]['text']}"


Creating en dict...


100%|██████████| 200000/200000 [00:08<00:00, 23499.66it/s]


In [259]:
en_dict

{'0_0': '<USER>: Hey, can you give me, like, three tips for staying healthy?\n<ASSISTANT>: Here are three tips for staying healthy: eat a balanced diet, stay hydrated by drinking plenty of water, and get at least seven hours of sleep each night.',
 '1_0': '<USER>: Hey, can you tell me what the three primary colors are?\n<ASSISTANT>: The three primary colors are red, blue, and yellow.',
 '2_0': '<USER>: So, can you, uh, describe the structure of an atom for me?\n<ASSISTANT>: An atom consists of three main parts: the nucleus, which is the center, made up of protons and neutrons, surrounded by electrons that orbit around it. Protons have a positive charge, neutrons have no charge, and electrons have a negative charge.',
 '3_0': '<USER>: Hey, can you tell me how we can, like, reduce air pollution?\n<ASSISTANT>: To reduce air pollution, use public transport, walk or bike, avoid burning fossil fuels, and recycle. Also, use energy-efficient appliances and turn them off when not in use.',
 '3_

In [ ]:
{ex['question']} {ex['answer']}
dic['']

In [6]:
def merge_pair(example):
    eid = f"{example['id'].split('_')[-1]}_{example['round']-1}"
    example["en"] = en_dict.get(eid, "")
    example["eu"] = f"{example['question']}\n<ASSISTANT>: {example['answer']}"
    return example

merged_dataset = eu_dataset.map(merge_pair)

In [7]:
merged_dataset[469]['eu']

'<USER>: Amaitu esaera hobe da segurua baino\n<ASSISTANT>: Sentitzen dugu, hobe da segurua izatea, segurua izatea baino.'

In [ ]:
# import multiprocess
# multiprocess.set_start_method("spawn", force=True)

In [54]:
scored_dataset = merged_dataset.shuffle().map(process)
# cleaned_dataset = scored_dataset.filter(lambda x: x['valid'])

Map:   0%|          | 0/420734 [00:00<?, ? examples/s]/home/andoni.sudupe/.conda/envs/omni/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Map:   1%|          | 3806/420734 [02:33<4:40:46, 24.75 examples/s] 


KeyboardInterrupt: 